In [6]:

# Final Script
# ─── asin_scraper.py ───────────────────────────────────────────────────────────
import time, random, csv, sys
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager


# ── CONFIG ────────────────────────────────────────────────────────────────────
EXCEL_INPUT_FILE = "Biswarup.xlsx"
ID_COLUMN_NAME   = "product_id"
OUT_BASENAME     = "amazon_product_ids_results"          # .xlsx / .csv produced
AMAZON_URL       = "https://www.amazon.in/dp/{}?th=1&psc=1&language=en_IN"
HEADLESS         = True
WAIT_TIME        = 5
PAUSE_RANGE      = (0, 1.5)      # polite delay between requests
# ───────────────────────────────────────────────────────────────────────────────

def make_driver(headless=True):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1200,900")
    opts.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()), options=opts)

def wait_txt(driver, selectors):
    """Return first non-empty text found for any CSS selector."""
    for css in selectors:
        try:
            elem = WebDriverWait(driver, WAIT_TIME).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, css)))
            txt = elem.text.strip()
            if txt:
                return txt
        except TimeoutException:
            pass
    return "N/A"

TITLE_SEL  = ["#productTitle", "span#title"]
PRICE_SEL  = [
    "#priceToPay .a-offscreen",
    "#corePriceDisplay_desktop_feature_div .a-offscreen",
    "#corePriceDisplay_desktop_feature_div span.a-price-whole",
    "#corePriceDisplay_desktop_feature_div span.a-price",
    "#priceblock_dealprice",
    "#priceblock_ourprice",
]
RATING_SEL = [
    "span[data-hook='rating-out-of-text']",
    "span.a-icon-alt"
]

RATING_COUNT_SEL = [
    "#acrCustomerReviewText",
    "span[data-hook='total-review-count']"
]

CATEGORY_SEL = [
    "#wayfinding-breadcrumbs_container a.a-link-normal",        # desktop breadcrumb links
    "#wayfinding-breadcrumbs_feature_div a.a-link-normal",
    "#wayfinding-breadcrumbs_feature_div li:not(.a-breadcrumb-divider)"
]

MRP_SEL = [
    "#listPriceValue",
    "#price .a-text-strike",
    "span.priceBlockStrikePriceString",
    "#priceblock_listprice",
    "#priceblock_ourprice.priceBlockStrikePriceString",
    "span.a-price.a-text-price > span.a-offscreen",
    "span.aok-relative span.aok-offscreen",
    "span.a-size-small.a-color-secondary.basisPrice",  # New entry for MRP
]

DISCOUNT_SEL = [
    "#regularprice_savings .a-color-price",
    "span[class*='savingsPercentage']",            # “-23%”
    "span[class*='priceToPay'] ~ span.a-color-price",  # “You Save”
    "td.a-span12.a-color-price",                   # legacy layout
]

def scrape_one(driver, asin):
    url = AMAZON_URL.format(asin)
    driver.get(url)
    try:
        driver.find_element(By.ID, "sp-cc-accept").click()
    except Exception:
        pass

    return {
        "ProductID": asin,
        "URL":       url,
        "Title":     wait_txt(driver, TITLE_SEL),
        "Selling Price":     wait_txt(driver, PRICE_SEL),
        "MRP":     wait_txt(driver, MRP_SEL),
        "Discount":     wait_txt(driver, DISCOUNT_SEL),
        "Rating":    wait_txt(driver, RATING_SEL),
        "Rating_Count":    wait_txt(driver, RATING_COUNT_SEL),
        "Category":  wait_txt(driver, CATEGORY_SEL),  # ← NEW
    }

# ── helpers ───────────────────────────────────────────────────────────────────
def save_partial(results):
    """Save DataFrame to both CSV and XLSX."""
    if not results:
        return
    df_out = pd.DataFrame(results)
    df_out.to_excel(f"{OUT_BASENAME}.xlsx", index=False)
    df_out.to_csv (f"{OUT_BASENAME}.csv",  index=False)
    print(f"  ↳ (autosaved {len(df_out)} rows)")

# ── main ──────────────────────────────────────────────────────────────────────
def main():
    df_in = pd.read_excel(EXCEL_INPUT_FILE)
    if ID_COLUMN_NAME not in df_in.columns:
        raise KeyError(f"Column “{ID_COLUMN_NAME}” not found in {EXCEL_INPUT_FILE}")
    asins = df_in[ID_COLUMN_NAME].dropna().astype(str).tolist()
    print(f"▶ Scraping {len(asins)} ASINs …")

    driver   = make_driver(HEADLESS)
    results  = []
    failures = []

    try:
        for i, asin in enumerate(asins, 1):
            try:
                print(f"[{i}/{len(asins)}] {asin} …", end=" ", flush=True)
                data = scrape_one(driver, asin)
                results.append(data)
                print("✓")
            except Exception as e_one:
                print(f"✗ ({e_one.__class__.__name__})")
                failures.append(asin)
            finally:
                save_partial(results)          # <- write after every loop
                time.sleep(random.uniform(*PAUSE_RANGE))
    finally:
        driver.quit()
        save_partial(results)                  # final save even on crash
        if failures:
            print("\n⚠ Failed ASINs:")
            for bad in failures:
                print("  -", bad)

    print(f"\n✔ Finished. Scraped {len(results)} products "
          f"({len(failures)} failures). Files: {OUT_BASENAME}.xlsx / .csv")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nInterrupted by user, exiting.", file=sys.stderr)


▶ Scraping 54 ASINs …
[1/54] B07PR1CL3S … ✓
  ↳ (autosaved 1 rows)
[2/54] B07PR1CL3S … ✓
  ↳ (autosaved 2 rows)
[3/54] B0BSLPSK47 … ✓
  ↳ (autosaved 3 rows)
[4/54] B09V7WS4PP … ✓
  ↳ (autosaved 4 rows)
[5/54] B071Z8M4KX … ✓
  ↳ (autosaved 5 rows)
[6/54] B0DFQ1R3W4 … ✓
  ↳ (autosaved 6 rows)
[7/54] B0CZ3ZPD8B … ✓
  ↳ (autosaved 7 rows)
[8/54] B0D8BZDPXB … ✓
  ↳ (autosaved 8 rows)
[9/54] B0DJW4X91W … ✓
  ↳ (autosaved 9 rows)
[10/54] B0DG92TB1B … ✓
  ↳ (autosaved 10 rows)
[11/54] B09SHWXWR8 … ✓
  ↳ (autosaved 11 rows)
[12/54] B0B1PXM75C … ✓
  ↳ (autosaved 12 rows)
[13/54] B0DHV38WWD … ✓
  ↳ (autosaved 13 rows)
[14/54] B084685MT1 … ✓
  ↳ (autosaved 14 rows)
[15/54] B0CCVGXFG7 … ✓
  ↳ (autosaved 15 rows)
[16/54] B071Z8M4KX … ✓
  ↳ (autosaved 16 rows)
[17/54] B0D6W7F7WH … ✓
  ↳ (autosaved 17 rows)
[18/54] B0DSZW4VQZ … ✓
  ↳ (autosaved 18 rows)
[19/54] B0F7RPKTN5 … ✓
  ↳ (autosaved 19 rows)
[20/54] B084685MT1 … ✓
  ↳ (autosaved 20 rows)
[21/54] B09XXW54QG … ✓
  ↳ (autosaved 21 rows)
[22/54] B